# CKKS Encoding
## 概要
前回は

$$
\sigma: z \in \mathbb{C}^{N} \rightarrow m(X) \in \mathbb{C}[X] / (X^N + 1) 
$$

というエンコードを考えていたが、行いたいのは、

$$
\sigma: z \in \mathbb{C}^{N/2} \rightarrow m(X) \in \mathbb{Z}[X] / (X^N + 1) 
$$

というエンコードである。

今回は上記のCKKSに使われるエンコードについて説明する


# 複素係数と整数係数の差を埋める
$\mathbb{C}^N$ は必ずしも整数係数にエンコードされるわけではない。
この問題を解決するために、cannonical embedding $\sigma$ について考える。

$ \mathcal{R} $ 上の多項式は整数係数であるが、複素数の根で考える必要がある。ここで、複素数の根の半分はもう半分の共役となっているはずである。  

$$
\sigma(\mathcal{R}) \subseteq \mathbb{H} \\
\mathbb{H} =  \lbrace (z_j)_{j \in Z^*_M} : z_{−j} = \overline{z_j} , \forall j \in Z^∗_M \rbrace \subseteq \mathbb{C}^N 
$$


例えば、8次円分多項式 $\Phi_8(X) = \prod_{gcd(8, i)=1} (x - \omega^i)$ は以下のようになる ($\omega$は1の原始8乗根)

![image](cyclotomic.jpeg)

8次円分多項式は、$\Phi_8(X) = (x-\omega^1) (x - \omega^3) (x - \omega^5) (x-\omega^7)$ という形になる。この根は上図のように、$\omega^1$ と $\overline{\omega^7}$, $\omega^3$ と $\overline{\omega^5}$ で互いに共役となっていることがわかる。  

このことから、整数係数に落とすには、共役の片方だけ考えてあげればよいことがわかる。

一般的には、 $X^N+1$ の根で実数の多項式を考えるため、任意の多項式において

$$
m(X) \in \mathbb{R}, \\
m(\xi^j) = m(\overline{\xi_{-j}})
$$

が得られる。

これより、任意の $\sigma(\mathcal{R})$ の要素は、NではなくN/2の次元となる。このことから、N/2の次元の複素数ベクトルをckksにおいてエンコードするためには、もう半分の共役となる根に複製して拡張する必要がある。 

 $\mathbb{H}$ の要素をとり、$\mathbb{C}^{N/2}$ に射影する処理を CKKS では $\pi$ と表す。

以上により $z \in \mathbb{C}^{N/2}$ を $\pi^{-1}$ により拡張し、$\pi^{-1}(z) \in \mathbb{H}$ を得ることができる。

In [10]:
import numpy as np

M = 8

def pi(z: np.array) -> np.array:
    """Projects a vector of H into C^{N/2}."""

    N = M // 4
    return z[:N]

def pi_inverse(z: np.array) -> np.array:
    """Expands a vector of C^{N/2} by expanding it with its
    complex conjugate."""

    z_conjugate = z[::-1]
    z_conjugate = [np.conjugate(x) for x in z_conjugate]
    return np.concatenate([z, z_conjugate])


In [14]:
z_in_H = pi_inverse(np.array([1 + 3j, 2+4j]))
z_in_H


array([1.+3.j, 2.+4.j, 2.-4.j, 1.-3.j])

In [15]:
pi(z_in_H)

array([1.+3.j, 2.+4.j])

## Coordinate-wise randomized rounding

問題点は、$\mathbb{H}$ (ただし、$\sigma(\mathcal{R}) \subseteq \mathbb{H} = z \in \mathcal{C}^N : z_j = \overline{z_{-j}}$) の要素が必ずしも $\sigma(\mathcal{R})$ に入るとは限らないため、

$$
\sigma : \mathcal{R} = \mathbb{Z}[X]/(X^N + 1) \rightarrow \sigma(\mathcal{R}) \subseteq \mathbb{H}
$$

を直接使用することができないことである。  
$\sigma$ は同型写像であるが、 $\mathcal{R}$ から $\sigma(\mathcal{R})$ の写像であり、 $\mathbb{H}$ への写像ではない。  
$\mathcal{R}$ が可算であることから、その同型写像である $\sigma(\mathcal{R})$ は可算である。しかし、$\mathbb{H}$ は $\mathbb{C}^{N/2}$ の同型であり、不可算であることが分かる。このことからも上記のことが分かる。  


先ほど、 $\pi$ を $\mathbb{H}$ から $\mathbb{C}^{N/2}$ への写像と定義したが、実際には $\pi^{-1}(z)$ が $\sigma(\mathcal{R})$ 上への写像である必要がある。  
このために、"coodinate-wise randomized rounding"(座標単位のランダム丸め)を用いる。  
これにより、実数xは $\lfloor x \rfloor$ か $\lfloor x \rfloor + 1$ に確率的に近い方に丸め込む。  

Randomized rounding: 基本的な考え方は、確率的方法を使用して、問題の緩和の最適な解決策を元の問題に対するほぼ最適な解決策に変換することです。(wikipedia)

$\mathcal{R}$ は 直行基底 $1, X, ..., X^{N-1}$ を持ち、 同型写像 $\sigma(\mathcal{R})$ も同様に、直行基底 $\beta = (b_1, b_2, ..., b_N) = (\sigma(1), \sigma(X), ..., \sigma(X^{N-1}))$ をもつ

このことから、任意の $z \in \mathbb{H}$ において、単純に $\beta$ に射影する

$$
z = \Sigma_{i=1}^N z_i b_i, z_i = \frac{<z, b_i>}{||b_i^2||} \in \mathbb{R}
$$

(正規直行基底ではないので規格化してからかけている)

$<x, y>$ はエルミート積 $\Sigma_{i=1}^N x_i \overline{y_i}$ を表す。  
このエルミート積は実数を出力する。なぜならば、$\mathbb{H}$に適用しているためである。 $\mathbb{H}$ と $\mathbb{R}^N$ の間の写像は等長同型写像であることからも、 $\mathbb{H}$での内積は実数を出力することが分かる  
(?)実数の座標に射影してるから、複素数を考えないのか？

これにより、 $z_i$ による座標が得られたので、 "coordinate-wise random rounding" によりランダムに近い整数に丸め込まれる。
$(\sigma(1), \sigma(X), ..., \sigma(X^{N-1}))$ を基底とする整数の座標上に多項式を得ることができたので、$\sigma(\mathbb{R})$に属することになる。

In [40]:
xi = np.exp(2 * np.pi * 1j / M)

# """Creates the basis (sigma(1), sigma(X), ..., sigma(X** N-1))."""
sigma_R_basis = np.array(CKKSEncoder.vandermonde(xi, M)).T


def compute_basis_coordinates(z):
    """Computes the coordinates of a vector with respect to the orthogonal lattice basis."""
    output = np.array([np.real(np.vdot(z, b) / np.vdot(b, b)) for b in sigma_R_basis])
    return output

def round_coordinates(coordinates):
    """Gives the integral rest."""
    coordinates = coordinates - np.floor(coordinates)
    return coordinates

def coordinate_wise_random_rounding(coordinates):
    """Rounds coordinates randonmly."""
    r = round_coordinates(coordinates)
    f = np.array([np.random.choice([c, c-1], 1, p=[1-c, c]) for c in r]).reshape(-1)

    rounded_coordinates = coordinates - f
    rounded_coordinates = [int(coeff) for coeff in rounded_coordinates]
    return rounded_coordinates

def sigma_R_discretization(z):
    """Projects a vector on the lattice using coordinate wise random rounding."""
    coordinates = compute_basis_coordinates(z)

    rounded_coordinates = coordinate_wise_random_rounding(coordinates)
    y = np.matmul(sigma_R_basis.T, rounded_coordinates)
    return y


In [32]:
sigma_R_basis


array([[ 1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ,
         1.00000000e+00+0.j        ,  1.00000000e+00+0.j        ],
       [ 7.07106781e-01+0.70710678j, -7.07106781e-01+0.70710678j,
        -7.07106781e-01-0.70710678j,  7.07106781e-01-0.70710678j],
       [ 2.22044605e-16+1.j        , -4.44089210e-16-1.j        ,
         1.11022302e-15+1.j        , -1.38777878e-15-1.j        ],
       [-7.07106781e-01+0.70710678j,  7.07106781e-01+0.70710678j,
         7.07106781e-01-0.70710678j, -7.07106781e-01-0.70710678j]])

In [41]:
z = np.array([1.2+3.j, 2.4+4.j, 2.8-4.j, 1.9])
coordinates = compute_basis_coordinates(z)
coordinates

array([ 2.075     ,  1.57331259, -1.25      ,  2.31577471])

In [36]:
rounded_coordinates = coordinate_wise_random_rounding(coordinates)
rounded_coordinates

array([ 2.,  2., -1.,  3.])

## 丸め誤差の対策
なお、丸め込みは、大きな精度のロスが生じる可能性がある。そこで、 $\Delta > 0$ をエンコード時にはかけ、デコード時に割ることで、 $1/\Delta$ という精度を保つ。

例えば、$x=1.4$を精度0.25で丸め込みたい場合は、$\Delta = 4$を設定する。
エンコードすると、

$$
\lfloor x \rfloor = \lfloor 4 * 1.4 \rfloor =  \lfloor 5.6 \rfloor = 6
$$

となり、デコード時に4で割ると、1.5となる

## 全体の流れ
1. $z \in \mathbb{C}^{N/2}$ から要素を受け取る
2. $\pi^{-1}(z) \in \mathbb{H}$ に拡張
3. $\Delta$ をかける
4. $\sigma(\mathcal{R}) : \lfloor \Delta \pi^{-1}(z) \rceil_{\sigma(\mathbb{R})} \in \sigma(\mathcal{R})$ に写像する
5. $\sigma : m(X) = \sigma^{-1} (\lfloor \Delta \pi^{-1}(z) \rceil_{\sigma(\mathbb{R})} \in \sigma(\mathcal{R}))$ エンコード 


$m(X)$ をデコードする際は、 $z = \pi \circ \sigma(\Delta^{-1} m(X)) $


In [2]:
import numpy as np

# パラメータの設定
M = 8
N = M//2

# \xiを設定する。
xi = np.exp(2 * np.pi * 1j / M)

In [47]:
from numpy.polynomial import Polynomial
# Encoderを定義する

scale = 4


class CKKSEncoder:
    """Complex vector を 多項式にエンコードする関数"""

    def __init__(self, M: int, scale: int):
        """エンコーダの初期化"""
        self.xi = np.exp(2 * np.pi * 1j / M)
        self.M = M
        self.N = M // 2
        self.create_sigma_R_basis()
        self.scale = scale


    @staticmethod
    def vandermonde(xi: np.complex128, M: int):
        """m乗根からVandermode matrixの計算 
        
        [
            [xi, xi^2, xi^3, ..., xi^N],
            [xi^3, x_i^(3*2), x_i^(3*3), ..., x_i^(3*N)],
            ...,
            [xi^N, x_i^(N*2), x_i^(N*3), ..., x_i^(N*N)]
        ]

        """
        N = M // 2
        matrix = []

        # それぞれのmatrixの列を生成する
        for i in range(N):
            # それぞれの列で異なる根を使う
            root = xi ** (2 * i + 1)
            row = []

            # それぞれの累乗を入れていく
            for j in range(N):
                row.append(root ** j)
            matrix.append(row)

        return matrix

    def sigma_inverse(self, b: np.array) -> Polynomial:
        """ベクトルbをM乗根の多項式に埋め込む
        """
        A = CKKSEncoder.vandermonde(self.xi, M)

        # A * bが多項式の係数になる
        coeffs = np.linalg.solve(A, b)

        p = Polynomial(coeffs)
        return p

    def sigma(self, p: Polynomial) -> np.array:
        """M乗根へ適用することで、多項式をデコードする"""

        outputs = []
        N = self.M // 2

        # シンプルに根の多項式に適用する
        for i in range(N):
            root = self.xi ** (2 * i + 1)
            output = p(root)
            outputs.append(output)
        return np.array(outputs)

    def pi(self, z: np.array) -> np.array:
        """Projects a vector of H into C^{N/2}."""
        
        N = self.M // 4
        return z[:N]

    def pi_inverse(self, z: np.array) -> np.array:
        """Expands a vector of C^{N/2} by expanding it with its
        complex conjugate."""
        
        z_conjugate = z[::-1]
        z_conjugate = [np.conjugate(x) for x in z_conjugate]
        return np.concatenate([z, z_conjugate])

    def create_sigma_R_basis(self):
        """Creates the basis (sigma(1), sigma(X), ..., sigma(X** N-1))."""

        self.sigma_R_basis = np.array(self.vandermonde(self.xi, self.M)).T
    
    def compute_basis_coordinates(self, z):
        """Computes the coordinates of a vector with respect to the orthogonal lattice basis."""
        output = np.array([np.real(np.vdot(z, b) / np.vdot(b,b)) for b in self.sigma_R_basis])
        return output

    def round_coordinates(self, coordinates):
        """Gives the integral rest."""
        coordinates = coordinates - np.floor(coordinates)
        return coordinates

    def coordinate_wise_random_rounding(self, coordinates):
        """Rounds coordinates randonmly."""
        r = self.round_coordinates(coordinates)
        f = np.array([np.random.choice([c, c-1], 1, p=[1-c, c]) for c in r]).reshape(-1)
        
        rounded_coordinates = coordinates - f
        rounded_coordinates = [int(coeff) for coeff in rounded_coordinates]
        return rounded_coordinates

    def sigma_R_discretization(self, z):
        """Projects a vector on the lattice using coordinate wise random rounding."""
        coordinates = self.compute_basis_coordinates(z)
        
        rounded_coordinates = self.coordinate_wise_random_rounding(coordinates)
        y = np.matmul(self.sigma_R_basis.T, rounded_coordinates)
        return y

    def encode(self, z: np.array) -> Polynomial:
        """Encodes a vector by expanding it first to H,
        scale it, project it on the lattice of sigma(R), and performs
        sigma inverse.
        """
        pi_z = self.pi_inverse(z)
        scaled_pi_z = self.scale * pi_z
        rounded_scale_pi_zi = self.sigma_R_discretization(scaled_pi_z)
        p = self.sigma_inverse(rounded_scale_pi_zi)
        
        # We round it afterwards due to numerical imprecision
        coef = np.round(np.real(p.coef)).astype(int)
        p = Polynomial(coef)
        return p

    def decode(self, p: Polynomial) -> np.array:
        """Decodes a polynomial by removing the scale, 
        evaluating on the roots, and project it on C^(N/2)"""
        rescaled_p = p / self.scale
        z = self.sigma(rescaled_p)
        pi_z = self.pi(z)
        return pi_z

In [57]:
scale = 32
M = 8
encoder = CKKSEncoder(M, scale)


In [58]:
z = np.array([3 + 4j, 2 - 1j])
z

array([3.+4.j, 2.-1.j])

In [59]:
p = encoder.encode(z)
p

Polynomial([80., 45., 80., 22.], domain=[-1,  1], window=[-1,  1])

In [60]:
encoder.decode(p)

array([3.008233+3.98050482j, 1.991767-1.01949518j])